In [2]:
import matplotlib.pyplot as plt
import json
import numpy as np
import random
import string
from collections import defaultdict
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn import linear_model
import gzip
import json
import csv
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from tqdm import tqdm

In [3]:
def parseData(path):
    for l in gzip.open(path, 'rt', encoding="utf8"):
        yield eval(l)

In [4]:
dataset = list(parseData("train_Category.json.gz"))

In [23]:
original_df = pd.DataFrame(dataset)

In [6]:
original_df.head()

,userID,genre,early_access,reviewID,hours,text,genreID,date,found_funny,user_id,compensation
0,u74382925,Adventure,False,r75487422,4.1,Short Review:\nA good starting chapter for thi...,3,2014-02-07,NaN,NaN,NaN
1,u76283959,Strategy,True,r47002764,235.1,I think this is on my top 3 favirote TCGs of a...,1,2017-05-04,NaN,NaN,NaN
2,u27203184,Action,False,r63957768,11.6,This is 10% suck\n20% skill\n15% concentrated ...,0,2015-12-30,794.0,NaN,NaN
3,u39188474,Adventure,False,r78559045,18.9,This game is realy realy nice to play. it's hi...,3,2015-07-19,1.0,NaN,NaN
4,u00273970,Action,False,r87488647,229.5,Los fans del erizo azul lo pidieron y aqui est...,0,2016-01-09,NaN,NaN,NaN


In [24]:
col = ['genreID', 'text', 'genre']
original_df = original_df[col]

In [ ]:
for d in train:
    t = d['text']
    t = t.lower() # lowercase string
    t = [c for c in t if not (c in punct)] # non-punct characters
    t = ''.join(t) # convert back to string
    words = t.strip().split() # tokenizes
    for w in words:
        #w = stemmer.stem(w)
        totalWords += 1
        wordCount[w] += 1

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/s1shen/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
english_stop_words = set(stopwords.words("english")) 
def message_cleaning(message):
    Test_punc_removed = [char for char in message if char not in string.punctuation]
    Test_punc_removed_join = ''.join(Test_punc_removed)
    Test_punc_removed_join_clean = [word for word in Test_punc_removed_join.split() if word.lower() not in english_stop_words]
    return ' '.join(Test_punc_removed_join_clean)

original_df['text'] = original_df['text'].apply(message_cleaning)

In [26]:
original_df.head()

,genreID,text,genre
0,3,Short Review good starting chapter series desp...,Adventure
1,1,think top 3 favirote TCGs time wish added draw...,Strategy
2,0,10 suck 20 skill 15 concentrated power 5 pleas...,Action
3,3,game realy realy nice play hide seek adult hor...,Adventure
4,0,Los fans del erizo azul lo pidieron aqui esta ...,Action


In [8]:
from io import StringIO
category_id_df = original_df[['genre', 'genreID']].drop_duplicates().sort_values('genreID')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['genreID', 'genre']].values)

In [9]:
testset = list(parseData("test_Category.json.gz"))
dftest = pd.DataFrame(testset)

In [33]:
dftest['text'] = dftest['text'].apply(message_cleaning)

In [40]:
split = 140000
df = original_df[:split]

In [41]:
df = original_df[:split]
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 3), stop_words='english')
features = tfidf.fit_transform(df.text).toarray()
labels = df.genreID
model = LinearSVC()
model.fit(features, labels)
text_features = tfidf.transform(dftest.text)
predictions = model.predict(text_features)

In [18]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(features, labels)
predictions = clf.predict(text_features)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [42]:
fp = open("predictions_Category_14000_tg.txt", 'w')
fp.write("userID-reviewID,prediction\n")
i = 0
for d in testset:
    fp.write(d['userID'] + '-' + d['reviewID'] + "," + str(predictions[i]) + "\n")
    i += 1

fp.close()
del tfidf, features, model, text_features, predictions